In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from numpy import percentile
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
hist_trans_df1 = pd.read_csv('../input/elo-cards-basic-eda-exposes-data-skewness/hist_DecFeb18.csv')

In [ ]:
hist_trans_df2 = pd.read_csv('../input/elo-cards-basic-eda-exposes-data-skewness/hist_SepNov.csv')

In [ ]:
new_trans_df=pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')

In [ ]:
trans_df = pd.concat([hist_trans_df1,hist_trans_df2,new_trans_df])

In [ ]:
train_df = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')

In [ ]:
train_df['first_active_month'] = pd.to_datetime(train_df['first_active_month'])

In [ ]:
test_df = pd.read_csv('../input/elo-merchant-category-recommendation/test.csv')

In [ ]:
test_df['first_active_month'] = pd.to_datetime(test_df['first_active_month'])

In [ ]:
train_df_merged = pd.merge(trans_df, train_df, how = 'inner', left_on = ['card_id'], right_on=['card_id'])

In [ ]:
test_df_merged = pd.merge(trans_df, test_df, how = 'inner', left_on = ['card_id'], right_on=['card_id'])

In [ ]:
merchant_df = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')

In [ ]:
merchant_df.head()

In [ ]:
train_df_merch = pd.merge(train_df_merged, merchant_df, how = 'inner', left_on = ['merchant_id'], right_on=['merchant_id'])

In [ ]:
test_df_merch = pd.merge(test_df_merged, merchant_df, how = 'inner', left_on = ['merchant_id'], right_on=['merchant_id'])

In [ ]:
train_df_merch.fillna(0,inplace=True)

In [ ]:
train_df_merged['purchase_date']=pd.to_datetime(train_df_merged['purchase_date'])

In [ ]:
test_df_merged['purchase_date']=pd.to_datetime(test_df_merged['purchase_date'])

In [ ]:
def gen_features(df,inp_df):
    month_lag_bycard = df.groupby(['card_id'])['month_lag'].sum().reset_index()
    purch_amt_bycard = df.groupby(['card_id'])['purchase_amount'].sum().reset_index()
    max_purch_dt = df.groupby(['card_id'])['purchase_date'].max().reset_index()
    insta_by_card = df.groupby(['card_id'])['installments'].sum().reset_index()
    #auth_ct_bycard=df.groupby('card_id')['authorized_flag'].apply(lambda x: (x=='Y').sum()).reset_index(name='Auth_count')
    df_comb = inp_df.merge(month_lag_bycard, how='left', on = ['card_id'])
    df_comb = df_comb.merge(purch_amt_bycard, how='left', on = ['card_id'])
    df_comb = df_comb.merge(max_purch_dt, how='left', on = ['card_id'])
    df_comb['age_of_card'] = (df_comb['purchase_date'] - df_comb['first_active_month']).dt.days
    df_comb = df_comb.merge(insta_by_card, how='left', on = ['card_id'])
    #train_df_comb = train_df_comb.merge(auth_ct_bycard, how='left', on = ['card_id'])
    df_comb.fillna(0,inplace=True)
    return df_comb

In [ ]:
def gen_merch_features(df, inp_df):
    sales_lag3_card = df.groupby(['card_id'])['avg_sales_lag3'].mean().reset_index()
    purch_lag3_card = df.groupby(['card_id'])['avg_purchases_lag3'].mean().reset_index()
    act_mth3_card = df.groupby(['card_id'])['active_months_lag3'].mean().reset_index()
    sales_lag6_card = df.groupby(['card_id'])['avg_sales_lag6'].mean().reset_index()
    purch_lag6_card = df.groupby(['card_id'])['avg_purchases_lag6'].mean().reset_index()
    act_mth6_card = df.groupby(['card_id'])['active_months_lag6'].mean().reset_index()
    sales_lag12_card = df.groupby(['card_id'])['avg_sales_lag12'].mean().reset_index()
    purch_lag12_card = df.groupby(['card_id'])['avg_purchases_lag12'].mean().reset_index()
    act_mth12_card = df.groupby(['card_id'])['active_months_lag12'].mean().reset_index()
    df_comb = inp_df.merge(sales_lag3_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(purch_lag3_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(act_mth3_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(sales_lag6_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(purch_lag6_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(act_mth6_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(sales_lag12_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(purch_lag12_card, how='left', on = ['card_id'])
    df_comb = df_comb.merge(act_mth12_card, how='left', on = ['card_id'])
    df_comb.fillna(0,inplace=True)
    return df_comb

In [ ]:
train_df_comb = gen_features(train_df_merged,train_df)
train_df_comb = gen_merch_features(train_df_merch,train_df_comb)

In [ ]:
len(train_df_comb)

In [ ]:
train_df_comb.to_csv('train_df_comb.csv')

In [ ]:
test_df_comb = gen_features(test_df_merged,test_df)
test_df_comb = gen_merch_features(test_df_merch,test_df_comb)

In [ ]:
len(test_df_comb)

In [ ]:
test_df_comb.loc[~(test_df_comb.purchase_amount==0)]

In [ ]:
test_df_comb.to_csv('test_df_comb.csv')